In [2]:
# Imports
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-ITGhy3LPfyvYrxbBci8pT3BlbkFJP5k2rqmOZhAdrOncxu9X"
os.environ["SERPER_API_KEY"] = "22c46d0058e5749e043b579d50860e011f4358ba"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai_tools import tool

In [3]:
url = 'https://www.sutterhealth.org/'

In [4]:
serper_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool(website_url=url)
website_search_rag_tool = WebsiteSearchTool()

In [5]:
# Information Specialist
information_specialist = Agent(
    role = 'Information Specialist',
    goal = 'Gather all information about a healthcare provider',
    backstory = 
        """You work for a digital healthcare company called as Healee.
        You are an expert in detailed research and data compilation.
        You have to find out all the important information possible about {client} healthcare provider.""",
    tools = [scrape_tool],
    verbose = True,
    allow_delegation=False
)

# Query Agent
search_query_generator = Agent(
  role='Search Query Generator',
  goal='Generate search queries to find similar healthcare providers companies to {client} based on provided criteria and gathered information',
  backstory='Proficient in creating effective search queries from given data.',
  allow_delegation=False
)

# Researcher Agent
researcher = Agent(
    role = 'Internet Researcher',
    goal = 'Find similar healthcare provider companies.',
    backstory = 
        """You work in the marketing team of a digital healthcare company.
        You are adept at using online resources to find relevant information quickly.
        "You are looking for potential healthcare provider clients similar to {client}
        that your company can pitch your platform to and sign them.""",
    tools=[serper_tool],
    allow_delegation=False
)

# Data Analyst Agent
# data_analyst = Agent(
#     role = 'Data Analyst', 
#     goal = 'Identify similarities among healthcare providers',
#     backstory =
#         """You work in the Data analysis team of a digital healthcare company.
#         You are skilled in analyzing data and identifying patterns.""",
#     verbose = True,
# )

In [6]:
# Tasks
# Client information gathering task
# information_gathering_task = Task(
#     description = "Gather detailed information about {client} healtcare provider given website link: {website_link}",
#     expected_output='A comprehensive report detailing all available information about the healthcare provider.',
#     tools=[serper_tool, scrape_tool],
#   agent=information_specialist
# )

# Research task 
# research_task = Task(
#   description='Search the internet to find healthcare providers similar to {client} healthcare provider.',
#   expected_output='A list of potential healthcare providers with brief profiles, their website links and contact information if available.',
#   tools=[serper_tool],
#   agent=researcher
# )

# Define the Analysis task
# analysis_task = Task(
#   description='Analyze the search results to find providers similar to the ones identified.',
#   expected_output='A report identifying the most similar healthcare providers.',
#   agent=data_analyst
# )

scrape_task = Task(
    description="""Scrape and extract all the relevant information about the company.""",
    agent=information_specialist,
    expected_output="""All the details about the {client} in a concise summary that can be used to build search queries based on the 
     search criteria of {search_criteria} to find similar companies.""")

generate_search_query_task = Task(
  description='Generate succinct search queries based on the provided criteria: {search_criteria} and information gathered by the Information Specialist.',
  expected_output='A list of search queries to be used by the Researcher Agent.',
  agent=search_query_generator
)

search_task = Task(
    description="""Search the internet using different queries provided by the Search Query Generator agent to find {quantity} companies 
    that the marketing team can pitch the Healee platform to. Provide a list of these companies, including a brief description of each 
    company and a link to their website.""",
    agent=researcher,
    expected_output="A list of potential companies including brief descriptions and website links."
)

In [10]:
# Create the crew
marketing_crew = Crew(
  agents=[information_specialist, search_query_generator, researcher],
  tasks=[scrape_task, generate_search_query_task, search_task],
  verbose=True
)

2024-07-03 15:21:49,479 - 8667892736 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [11]:
# Kickoff the crew with the input for finding similar healthcare providers
inputs = {'client': 'Sutter Health', 'search_criteria': 'location and speciality', 'quantity': 10}
result = marketing_crew.kickoff(inputs=inputs)

 [2024-07-03 15:21:49][DEBUG]: == Working Agent: Information Specialist
 [2024-07-03 15:21:49][INFO]: == Starting Task: Scrape and extract all the relevant information about the company.


> Entering new CrewAgentExecutor chain...
I need to gather comprehensive information about Sutter Health from their official website. The best approach is to read and extract relevant content directly from the site.

Action: Read website content
Action Input: {"url": "https://www.sutterhealth.org/"} 

Sutter Health | Doctors and Hospitals | Northern California
Error was Detected
 						Your browser doesn't support JavaScript code, or you have disabled JavaScript.
Jump to content
Set Your Location
Sign in or Enroll
Set Your LocationSet Your Location
Find a Doctor
Find a Location
Donate Now
Patient Login
Sign In to My Health Online
Enroll in My Health Online
Learn About My Health Online
Enter a search term:
View all resultsfor 
Enter City or ZipGeolocateMake this my locationOpen 													Choose a m

In [12]:
print(result)

### Potential Healthcare Provider Clients in Northern California

1. **UCSF Medical Center**
   - **Description**: UCSF Medical Center is recognized for innovative treatments, advanced technology, collaboration among clinicians and scientists, and a highly personalized approach to patient care.
   - **Website**: [UCSF Medical Center](https://www.ucsfhealth.org/)

2. **Stanford Health Care**
   - **Description**: Stanford Health Care seeks to provide comprehensive patient care and is known for its advanced medical treatments and breakthrough research.
   - **Website**: [Stanford Health Care](https://stanfordhealthcare.org/)

3. **Providence St. Joseph Health**
   - **Description**: This health system operates a large network of hospitals and clinics providing a wide range of healthcare services, including primary, specialty, and urgent care.
   - **Website**: [Providence St. Joseph Health](https://www.providence.org/)

4. **Adventist Health**
   - **Description**: Adventist Health offer